In [32]:
# unicode, regex, json for text digestion
import sys
import unicodedata
import re
import json
import prepare
sys.path.append('..')

# nltk: natural language toolkit -> tokenization, stopwords (more on this soon)
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import acquire1
# pandas dataframe manipulation, acquire script, time formatting
import pandas as pd
from time import strftime

# shh, down in front
import warnings
warnings.filterwarnings('ignore')

In [30]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re

def get_endpoints():
    '''This function scrapes repository collections on github.com and returns a list 
    of url endpoints for those repositories.
    
    To use these endpoints, append each string value to 'https://github.com' to
    create a list of urls '''
    # create an empty list to store endpoints
    endpoints = []
    # go to each url - trending repos daily, weekly, and monthly
    for url in ['https://github.com/trending?since=daily&spoken_language_code=en',
                'https://github.com/trending?since=weekly&spoken_language_code=en',
                'https://github.com/trending?since=monthly&spoken_language_code=en']:
        # get the response
        response = get(url)
        # create the beautiful soup object
        soup = BeautifulSoup(response.text, 'html.parser')
        # identify html objects containing each repository
        for repo in soup.select('.Box-row'):
            # pull out the url endpoint for that repo and append to the list
            endpoints.append(repo
                             .select_one('h1')
                             .select_one('a')
                             .attrs['href'])
    # head to a new set of repo collections and repeat the process
    for url in ['https://github.com/collections/learn-to-code',
                'https://github.com/collections/open-source-organizations']:
        response = get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for repo in soup.select('article'):
            endpoints.append(repo
                                .select_one('h1')
                                .select_one('a')
                                .attrs['href'])
    # return the list of endpoints
    return endpoints

In [34]:
df = pd.read_json('data1.json')
df = prepare.prep_repos(df)
df

,repo,language,original,clean,stemmed,lemmatized,language_reduced
0,google/googletest,C++,# GoogleTest\n\n### Announcements\n\n#### Live...,googletest announcements live head googletest ...,googletest announc live head googletest follow...,googletest announcement live head googletest f...,Other
1,projectdiscovery/nuclei-templates,Python,"\n\n<h1 align=""center"">\nNuclei Templates\n</h...",h1 aligncenter nuclei templates h1 h4 aligncen...,h1 aligncent nuclei templat h1 h4 aligncenterc...,h1 aligncenter nucleus template h1 h4 aligncen...,Python
2,digitalocean/nginxconfig.io,JavaScript,[![GitHub stars](https://img.shields.io/github...,github stars https imgshieldsio github stars d...,github star http imgshieldsio github star digi...,github star http imgshieldsio github star digi...,JavaScript
3,flutter/flutter,Dart,# [![Flutter logo][]][flutter.dev]\n\n[![Build...,flutter logo flutterdev build status cirrus bu...,flutter logo flutterdev build statu cirru buil...,flutter logo flutterdev build status cirrus bu...,Other
4,PaddlePaddle/PaddleOCR,Python,"English | [简体中文](README_ch.md)\n\n<p align=""ce...",english readmechmd p aligncenter img src doc p...,english readmechmd p aligncent img src doc pad...,english readmechmd p aligncenter img src doc p...,Python
...,...,...,...,...,...,...,...
104,cloudflare/cloudflare.github.io,CSS,cloudflare.github.io\n====================\n\n...,cloudflaregithubio source code https cloudflar...,cloudflaregithubio sourc code http cloudflareg...,cloudflaregithubio source code http cloudflare...,Other
105,eleme/eleme.github.io,CSS,# Team Blog\n\nRequirements to join this blog:...,team blog requirements join blog 1 must develo...,team blog requir join blog 1 must develop elem...,team blog requirement join blog 1 must develop...,Other
106,didi/didi.github.io,HTML,# didi.github.io\ndidi website\n,didigithubio didi website,didigithubio didi websit,didigithubio didi website,HTML
107,alibaba/alibaba.github.com,CSS,,,,,Other


In [35]:
df[df.language.isna()]

,repo,language,original,clean,stemmed,lemmatized,language_reduced
6,felipefialho/frontend-challenges,None,"<p align=""center""><img src=""https://user-image...",p aligncenterimg srchttps userimagesgithubuser...,p aligncenterimg srchttp userimagesgithubuserc...,p aligncenterimg srchttps userimagesgithubuser...,Other
8,florinpop17/app-ideas,None,## :ledger: App Ideas Collection\n\n![App Idea...,ledger app ideas collection app ideas image ap...,ledger app idea collect app idea imag appideas...,ledger app idea collection app idea image appi...,Other
21,ACM-Alexandria-SC/Internships,None,\n\n\n\n# Summer 2022 Tech Internships for Egy...,summer 2022 tech internships egyptian students...,summer 2022 tech internship egyptian student c...,summer 2022 tech internship egyptian student c...,Other
36,onceupon/Bash-Oneliner,None,# Bash-Oneliner\nI am glad that you are here! ...,bashoneliner glad working bioinformatics years...,bashonelin glad work bioinformat year ago amaz...,bashoneliner glad working bioinformatics year ...,Other
75,datasciencemasters/go,None,_created & maintained by [@clarecorthell](http...,created maintained clarecorthell http bitly cl...,creat maintain clarecorthel http bitli clareco...,created maintained clarecorthell http bitly cl...,Other
76,practical-tutorials/project-based-learning,None,# Project Based Learning\n\n[![Gitter](https:/...,project based learning gitter https badgesgitt...,project base learn gitter http badgesgitterim ...,project based learning gitter http badgesgitte...,Other
77,zhiwehu/Python-programming-exercises,None,# Python-programming-exercises\n\n100+ Python ...,pythonprogrammingexercises 100 python challeng...,pythonprogrammingexercis 100 python challeng p...,pythonprogrammingexercises 100 python challeng...,Other
78,MunGell/awesome-for-beginners,None,[![Doppler](assets/doppler.png)](https://www.d...,doppler assets dopplerpng https wwwdopplercom ...,doppler asset dopplerpng http wwwdopplercom ut...,doppler asset dopplerpng http wwwdopplercom ut...,Other
79,appacademy/welcome-to-open,None,# Welcome to a/A Open\n\n![](open-image.png)\n...,welcome open openimagepng tldr click learn cod...,welcom open openimagepng tldr click learn code...,welcome open openimagepng tldr click learn cod...,Other
84,karan/Projects,None,Mega Project List\r\n========\r\n\r\nA list of...,mega project list list practical projects anyo...,mega project list list practic project anyon s...,mega project list list practical project anyon...,Other
